In [69]:
import pandas as pd
import numpy as np

In [70]:
test = pd.read_csv("testbc7185d.csv")

In [71]:
df_calls = pd.read_csv("callsf0d4f5a.csv")

In [72]:
df_reasons = pd.read_csv("reason18315ff.csv")

In [73]:
test = pd.merge(test, df_calls, on='call_id', how='inner') 

In [74]:
train = pd.merge(df_calls, df_reasons, on='call_id', how='inner') 

In [75]:
train.head()

,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,call_transcript,primary_call_reason
0,4667960400,2033123310,963118,7/31/2024 23:56,8/1/2024 0:03,8/1/2024 0:34,\n\nAgent: Thank you for calling United Airlin...,Voluntary Cancel
1,1122072124,8186702651,519057,8/1/2024 0:03,8/1/2024 0:06,8/1/2024 0:18,\n\nAgent: Thank you for calling United Airlin...,Booking
2,6834291559,2416856629,158319,7/31/2024 23:59,8/1/2024 0:07,8/1/2024 0:26,\n\nAgent: Thank you for calling United Airlin...,IRROPS
3,2266439882,1154544516,488324,8/1/2024 0:05,8/1/2024 0:10,8/1/2024 0:17,\n\nAgent: Thank you for calling United Airlin...,Upgrade
4,1211603231,5214456437,721730,8/1/2024 0:04,8/1/2024 0:14,8/1/2024 0:23,\n\nAgent: Thank you for calling United Airlin...,Seating


In [76]:
test.head()

,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,call_transcript
0,7732610078,4029953261,488324,8/1/2024 0:23,8/1/2024 0:34,8/1/2024 1:32,\n\nAgent: Thank you for calling United Airlin...
1,2400299738,2034858976,963118,8/1/2024 1:33,8/1/2024 1:41,8/1/2024 1:54,\n\nAgent: Thank you for calling United Airlin...
2,6533095063,1874845993,519057,8/1/2024 2:17,8/1/2024 2:27,8/1/2024 2:29,\n\nAgent: Thank you for calling United Airlin...
3,7774450920,2633426588,963118,8/1/2024 2:49,8/1/2024 2:53,8/1/2024 2:57,\n\nAgent: Thank you for calling United Airlin...
4,9214147168,8334856492,158319,8/1/2024 2:49,8/1/2024 2:55,8/1/2024 2:59,\n\nAgent: Thank you for calling United Airlin...


In [77]:
train.columns

Index(['call_id', 'customer_id', 'agent_id', 'call_start_datetime',
       'agent_assigned_datetime', 'call_end_datetime', 'call_transcript',
       'primary_call_reason'],
      dtype='object')

In [78]:
train.drop([ 'customer_id', 'agent_id', 'call_start_datetime',
       'agent_assigned_datetime', 'call_end_datetime'], axis = 1, inplace = True)

In [79]:
test.drop([ 'customer_id', 'agent_id', 'call_start_datetime',
       'agent_assigned_datetime', 'call_end_datetime'], axis = 1, inplace = True)

In [80]:
train = train.dropna(subset=['primary_call_reason'], axis=0)


In [81]:
train.shape

(66653, 3)

In [82]:
train.head()

,call_id,call_transcript,primary_call_reason
0,4667960400,\n\nAgent: Thank you for calling United Airlin...,Voluntary Cancel
1,1122072124,\n\nAgent: Thank you for calling United Airlin...,Booking
2,6834291559,\n\nAgent: Thank you for calling United Airlin...,IRROPS
3,2266439882,\n\nAgent: Thank you for calling United Airlin...,Upgrade
4,1211603231,\n\nAgent: Thank you for calling United Airlin...,Seating


In [83]:
test.head()

,call_id,call_transcript
0,7732610078,\n\nAgent: Thank you for calling United Airlin...
1,2400299738,\n\nAgent: Thank you for calling United Airlin...
2,6533095063,\n\nAgent: Thank you for calling United Airlin...
3,7774450920,\n\nAgent: Thank you for calling United Airlin...
4,9214147168,\n\nAgent: Thank you for calling United Airlin...


In [84]:
def clean_text(text):
    text = text.replace('-', ' ')
    text = ' '.join(text.split())
    text = text.replace('and', '&')
    return text


In [85]:
train['primary_call_reason'] = train['primary_call_reason'].apply(clean_text)

In [86]:
from sklearn.model_selection import train_test_split
train['call_transcript'] = train['call_transcript'].str.strip()

X_train, X_val, y_train, y_val = train_test_split(train['call_transcript'], 
                                                  train['primary_call_reason'], 
                                                  test_size=0.2, 
                                                  random_state=42,
                                                  stratify=train['primary_call_reason'])  # Stratify by the target column

In [87]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)

class_names = label_encoder.classes_


In [88]:
print(len(class_names))

20


In [89]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [90]:
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_val_tokens = tokenizer.texts_to_sequences(X_val)

max_length = 128
X_train_padded = pad_sequences(X_train_tokens, maxlen=max_length, padding='post', truncating='post')
X_val_padded = pad_sequences(X_val_tokens, maxlen=max_length, padding='post', truncating='post')

In [91]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=64, input_length=128),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

model.summary()



c:\PythonPersonal\Extras\myvenv\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [92]:
history = model.fit(X_train_padded, y_train, 
                    epochs=5, 
                    validation_data=(X_val_padded, y_val),
                    batch_size=32)

Epoch 1/5
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 71s 41ms/step - accuracy: 0.1881 - loss: 2.6141 - val_accuracy: 0.2106 - val_loss: 2.5382
Epoch 2/5
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 72s 43ms/step - accuracy: 0.2192 - loss: 2.5292 - val_accuracy: 0.2158 - val_loss: 2.5306
Epoch 3/5
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 72s 43ms/step - accuracy: 0.2262 - loss: 2.5037 - val_accuracy: 0.2211 - val_loss: 2.5161
Epoch 4/5
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 70s 42ms/step - accuracy: 0.2328 - loss: 2.4843 - val_accuracy: 0.2181 - val_loss: 2.5244
Epoch 5/5
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 74s 44ms/step - accuracy: 0.2396 - loss: 2.4565 - val_accuracy: 0.2188 - val_loss: 2.5352


In [93]:
val_loss, val_acc = model.evaluate(X_val_padded, y_val)
print(f"Validation Accuracy: {val_acc:.4f}")


417/417 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.2157 - loss: 2.5350
Validation Accuracy: 0.2188


In [94]:
print(X_train)
print(y_train)
print(X_val)
print(y_val)

6915     Agent: Thank you for calling United Airlines, ...
32375    Agent: Thank you for calling United Airlines c...
4922     Agent: Thank you for calling United Airlines c...
66056    Agent: Thank you for calling United Airlines, ...
2995     Agent: Thank you for calling United Airlines, ...
                               ...                        
50351    Agent: Thank you for calling United Airlines c...
29091    Agent: Thank you for calling United Airlines, ...
40945    Agent: Thank you for calling United Airlines c...
38138    Agent: Thank you for calling United Airlines, ...
24505    Agent: Thank you for calling United Airlines, ...
Name: call_transcript, Length: 53322, dtype: object
[13  4  8 ...  1 12  9]
62897    Agent: Thank you for calling United Airlines, ...
34373    Agent: Thank you for calling United Airlines c...
14224    Agent: Thank you for calling United Airlines, ...
64763    Agent: Thank you for calling United Airlines, ...
37384    Agent: Thank you for calling U

In [95]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words[:50])  

X_train_truncated = X_train.apply(preprocess_text)
X_val_truncated = X_val.apply(preprocess_text)

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def embed_in_batches(data, batch_size=32):
    embeddings = []
    for i in range(0, len(data), batch_size):
        batch = data[i:i + batch_size]
        batch_embeddings = embed(batch.tolist())
        embeddings.append(batch_embeddings)
    return np.concatenate(embeddings)

X_train_embeddings = embed_in_batches(X_train_truncated)
X_val_embeddings = embed_in_batches(X_val_truncated)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shaun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [96]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(512,)),  
    tf.keras.layers.Dense(512, activation='relu'),   
    tf.keras.layers.Dense(256, activation='relu'),   
    tf.keras.layers.Dense(128, activation='relu'),    
    tf.keras.layers.Dense(20, activation='softmax')    
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(X_train_embeddings, y_train, 
          validation_data=(X_val_embeddings, y_val), 
          epochs=20, 
          batch_size=64,  
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

Epoch 1/20


c:\PythonPersonal\Extras\myvenv\lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


834/834 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.1956 - loss: 2.6024 - val_accuracy: 0.1939 - val_loss: 2.5657
Epoch 2/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2070 - loss: 2.5587 - val_accuracy: 0.2037 - val_loss: 2.5551
Epoch 3/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2078 - loss: 2.5444 - val_accuracy: 0.2085 - val_loss: 2.5498
Epoch 4/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2133 - loss: 2.5326 - val_accuracy: 0.2109 - val_loss: 2.5481
Epoch 5/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2149 - loss: 2.5364 - val_accuracy: 0.2132 - val_loss: 2.5445
Epoch 6/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2193 - loss: 2.5191 - val_accuracy: 0.2099 - val_loss: 2.5528
Epoch 7/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2218 - loss: 2.5183 - val_accuracy: 0.2146 - val_loss: 2.5371
Epoch 8/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2217 - loss: 2.5047 - val_accuracy: 0.2111 - val_

In [97]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(X_val_embeddings, y_val)
print(f'Validation Accuracy: {val_accuracy}')

417/417 ━━━━━━━━━━━━━━━━━━━━ 0s 932us/step - accuracy: 0.2107 - loss: 2.5448
Validation Accuracy: 0.2137123942375183


In [98]:
X_train_embeddings = np.expand_dims(X_train_embeddings, axis=1)
X_val_embeddings = np.expand_dims(X_val_embeddings, axis=1)

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1, 512)),  
    tf.keras.layers.LSTM(128, return_sequences=False), 
    tf.keras.layers.Dense(128, activation='relu'),     
    tf.keras.layers.Dense(20, activation='softmax')   
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])


model.fit(X_train_embeddings, y_train, 
          validation_data=(X_val_embeddings, y_val), 
          epochs=20, 
          batch_size=64,  
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])  

Epoch 1/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.1970 - loss: 2.6103 - val_accuracy: 0.2045 - val_loss: 2.5592
Epoch 2/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2066 - loss: 2.5552 - val_accuracy: 0.2066 - val_loss: 2.5461
Epoch 3/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2111 - loss: 2.5405 - val_accuracy: 0.2075 - val_loss: 2.5513
Epoch 4/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2120 - loss: 2.5333 - val_accuracy: 0.2090 - val_loss: 2.5457
Epoch 5/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2142 - loss: 2.5363 - val_accuracy: 0.2117 - val_loss: 2.5433
Epoch 6/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2160 - loss: 2.5300 - val_accuracy: 0.2127 - val_loss: 2.5379
Epoch 7/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2181 - loss: 2.5172 - val_accuracy: 0.2127 - val_loss: 2.5329
Epoch 8/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2187 - loss: 2.5116 - val_accuracy: 0.

In [99]:
val_loss, val_accuracy = model.evaluate(X_val_embeddings, y_val)
print(f'Validation Accuracy: {val_accuracy}')


417/417 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2114 - loss: 2.5356
Validation Accuracy: 0.213412344455719


In [100]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1, 512)),  
    tf.keras.layers.LSTM(256, return_sequences=True),  
    tf.keras.layers.Dropout(0.3),                      
    tf.keras.layers.LSTM(128, return_sequences=False), 
    tf.keras.layers.Dropout(0.3),                      
    tf.keras.layers.Dense(256, activation='relu'),    
    tf.keras.layers.Dense(128, activation='relu'),     
    tf.keras.layers.Dense(20, activation='softmax')  
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),  
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()




Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 1, 256)         │       787,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 1, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 128)            │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_55 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_56 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_57 (Dense)                │ (None, 20)             │         2,580 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,053,076 (4.02 MB)

 Trainable params: 1,053,076 (4.02 MB)

 Non-trainable params: 0 (0.00 B)

In [101]:
history = model.fit(X_train_embeddings, y_train, 
                    validation_data=(X_val_embeddings, y_val), 
                    epochs=30,                        
                    batch_size=64,                        
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])  

Epoch 1/30
834/834 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.1929 - loss: 2.6311 - val_accuracy: 0.1997 - val_loss: 2.5736
Epoch 2/30
834/834 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.2006 - loss: 2.5628 - val_accuracy: 0.2052 - val_loss: 2.5567
Epoch 3/30
834/834 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.2036 - loss: 2.5561 - val_accuracy: 0.2038 - val_loss: 2.5524
Epoch 4/30
834/834 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.2056 - loss: 2.5526 - val_accuracy: 0.2037 - val_loss: 2.5516
Epoch 5/30
834/834 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.2036 - loss: 2.5461 - val_accuracy: 0.2072 - val_loss: 2.5464
Epoch 6/30
834/834 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.2097 - loss: 2.5440 - val_accuracy: 0.2082 - val_loss: 2.5434
Epoch 7/30
834/834 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.2097 - loss: 2.5385 - val_accuracy: 0.2109 - val_loss: 2.5403
Epoch 8/30
834/834 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.2152 - loss: 2.5357 - val_accuracy: 0.

In [102]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(X_val_embeddings, y_val)
print(f'Validation Accuracy: {val_accuracy}')

417/417 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.2132 - loss: 2.5371
Validation Accuracy: 0.217087984085083


In [103]:
test.shape

(5157, 2)

In [104]:
test.head()

,call_id,call_transcript
0,7732610078,\n\nAgent: Thank you for calling United Airlin...
1,2400299738,\n\nAgent: Thank you for calling United Airlin...
2,6533095063,\n\nAgent: Thank you for calling United Airlin...
3,7774450920,\n\nAgent: Thank you for calling United Airlin...
4,9214147168,\n\nAgent: Thank you for calling United Airlin...


In [106]:
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def preprocess_transcripts(transcripts):
    processed_transcripts = []
    for transcript in transcripts:
        words = re.findall(r'\b\w+\b', transcript)
        filtered_words = [word for word in words if word.lower() not in stop_words]
        truncated_words = filtered_words[:50]
        processed_transcripts.append(' '.join(truncated_words))
    return processed_transcripts

test_transcripts = preprocess_transcripts(test['call_transcript'])

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def embed_in_batches(data, batch_size=32):
    embeddings = []
    for i in range(0, len(data), batch_size):
        batch = data[i:i + batch_size]
        batch_embeddings = embed(batch)
        embeddings.append(batch_embeddings)
    return np.concatenate(embeddings)

X_test_embeddings = embed_in_batches(test_transcripts)

X_test_embeddings = np.expand_dims(X_test_embeddings, axis=1)  

predictions = model.predict(X_test_embeddings)
predicted_primary_call_reason = np.argmax(predictions, axis=1)
test['predicted_primary_call_reason'] = predicted_primary_call_reason

print(test[['call_id', 'predicted_primary_call_reason']].head())


162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
      call_id  predicted_primary_call_reason
0  7732610078                              8
1  2400299738                              8
2  6533095063                              8
3  7774450920                              8
4  9214147168                              8


In [107]:
predicted_call_types = label_encoder.inverse_transform(predicted_primary_call_reason)

predictions_df = pd.DataFrame({
    'call_id': test['call_id'],
    'primary_call_reason': predicted_call_types
})


In [108]:
predictions_df.head(20)

,call_id,primary_call_reason
0,7732610078,IRROPS
1,2400299738,IRROPS
2,6533095063,IRROPS
3,7774450920,IRROPS
4,9214147168,IRROPS
5,2931134074,Voluntary Change
6,2010588624,IRROPS
7,6611494442,Voluntary Change
8,8655666048,IRROPS
9,3564872843,IRROPS


In [109]:
predictions_df.to_csv('test_ChanduChampion.csv', index=False)